In [1]:
import stumpy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [3]:
T_A = np.array([1, 0, 3, 9, 2, 1, 1, 15, 3, 14, 2, 10, 7]).astype(np.float64)
# T_A = np.array([1, 0, 3, 9, 2, 1, 1, 3, 6, 2]).astype(np.float64)
# T_B = np.array([1, 0, 3, 9, 2, 1, 1, 15]).astype(np.float64)
m = 3
d = 2

mp_orig = stumpy.stump_dil(T_A=T_A, m=m, d=d)
print(mp_orig)

/vol/fob-vol3/mi20/hirschmi/mp_dilation_master/stumpy_master_mp_dilation/stumpy/core.py:67: UserWarning: Excluding `['normalize', 'p']` parameters, function `stump_dil(['T_A', 'm', 'T_B', 'ignore_trivial', 'k', 'd']) and function `aamp(['T_A', 'm', 'T_B', 'ignore_trivial', 'k']) have different arguments/parameters.
  warnings.warn(msg)


[[0.0 6 -1 6]
 [0.725621964223909 6 0 5]
 [0.5749199116934414 7 -1 7]
 [0.1424943043046124 4 2 7]
 [0.1424943043046124 3 0 8]
 [0.7889556659397083 6 0 -1]
 [0.0 0 0 7]
 [0.5749199116934414 2 2 -1]
 [0.574919911693439 4 4 -1]]


In [2]:
T_A = np.array([1, 2, 3, 0, 3, 9, 2, 1, 2, 3, 1, 15, 3, 14, 2, 10, 7]).astype(np.float64)
m = 3
mp = stumpy.stump(T_A=T_A, m=m)
print(mp)

d=1
mp_dil = stumpy.stump_dil(T_A=T_A, m=m, d=d)
print(mp_dil)

[[0.0 7 -1 7]
 [0.3288114408414133 8 -1 8]
 [0.0 6 0 6]
 [0.3288114408414133 0 0 7]
 [0.09921547503779264 12 1 12]
 [1.3946067503040966 13 2 13]
 [0.0 2 2 11]
 [0.0 0 0 9]
 [0.3288114408414133 1 1 12]
 [0.7997306517513353 3 3 11]
 [0.3589616108906294 12 4 12]
 [0.13015840458292768 6 6 13]
 [0.09921547503779264 4 4 14]
 [0.4461606220875056 11 11 -1]
 [0.9147863787458559 10 10 -1]]


/vol/fob-vol3/mi20/hirschmi/mp_dilation_master/stumpy_master_mp_dilation/stumpy/core.py:67: UserWarning: Excluding `['normalize', 'p']` parameters, function `stump_dil(['T_A', 'm', 'T_B', 'ignore_trivial', 'k', 'd']) and function `aamp(['T_A', 'm', 'T_B', 'ignore_trivial', 'k']) have different arguments/parameters.
  warnings.warn(msg)


[[0.0 7 -1 7]
 [0.3288114408414133 8 -1 8]
 [0.0 6 0 6]
 [0.3288114408414133 0 0 7]
 [0.09921547503779264 12 1 12]
 [1.3946067503040966 13 2 13]
 [0.0 2 2 11]
 [0.0 0 0 9]
 [0.3288114408414133 1 1 12]
 [0.7997306517513353 3 3 11]
 [0.3589616108906294 12 4 12]
 [0.13015840458292768 6 6 13]
 [0.09921547503779264 4 4 14]
 [0.4461606220875056 11 11 -1]
 [0.9147863787458559 10 10 -1]]


In [2]:

# Arguments:
ts = np.array([1, 0, 3, 9, 2, 1, 1, 15, 3, 14, 2, 10, 7]).astype(np.float64)
index_original = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12])
window_size = 3
d = 2

# mp_orig = stumpy.stump(ts, m=window_size, ignore_trivial=False, d=d)

def dilation_mapping(X: np.ndarray, d: int) -> np.ndarray:
    result = X[0::d]
    for i in range(1, d):
        next = X[i::d]
        result = np.concatenate((result, next))
    return result


# 1. TS -dilationMaping-> dilated TS
ts_d = dilation_mapping(ts, d)
index_dilated = dilation_mapping(index_original, d)
print(f"TS: {ts}")
print(f"Index: {index_original}")
print(f"TS dilated: {ts_d}")
print(f"Index dilated: {index_dilated}")

# 2. dilated TS, m -stumpy-> MP 
mp_dilated_unfixed = stumpy.stump(ts_d, m=window_size)
print(f"MP dilated unfixed:  {mp_dilated_unfixed}")

# 3. MP, NNIndex -dilationMapping-> fixedMP                            muss hier auch MP nochmal gemappt werden?
mp_dilated_fixed = dilation_mapping(mp_dilated_unfixed, d)
index_dilated_fixed = dilation_mapping(index_dilated, d)
print(f"Index dilated fixed: {index_dilated_fixed}")
print(f"MP dilated fixed: {mp_dilated_fixed}")

# 4. Test Use Cases
# ...



# mp[mp_idx, 0] -> Euclidean Distance
# mp[mp_idx, 1] -> Index Nearest Neighbor
# mp[mp_idx, 2] -> Index “left” Nearest Neighbor
# mp[mp_idx, 3] -> Index “right” Nearest Neighbor

TS: [ 1.  0.  3.  9.  2.  1.  1. 15.  3. 14.  2. 10.  7.]
Index: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
TS dilated: [ 1.  3.  2.  1.  3.  2.  7.  0.  9.  1. 15. 14. 10.]
Index dilated: [ 0  2  4  6  8 10 12  1  3  5  7  9 11]
MP dilated unfixed:  [[0.0 3 -1 3]
 [0.5749199116934414 10 -1 10]
 [0.1424943043046124 8 0 8]
 [0.0 0 0 7]
 [0.574919911693439 2 2 8]
 [0.6585969196219232 7 3 7]
 [0.39420922977028316 8 2 8]
 [0.6585969196219232 5 5 9]
 [0.1424943043046124 2 2 10]
 [0.7889556659397083 3 3 -1]
 [0.5749199116934414 1 1 -1]]
Index dilated fixed: [ 0  4  8 12  3  7 11  2  6 10  1  5  9]
MP dilated fixed: [[0.0 3 -1 3]
 [0.1424943043046124 8 0 8]
 [0.574919911693439 2 2 8]
 [0.39420922977028316 8 2 8]
 [0.1424943043046124 2 2 10]
 [0.5749199116934414 1 1 -1]
 [0.5749199116934414 10 -1 10]
 [0.0 0 0 7]
 [0.6585969196219232 7 3 7]
 [0.6585969196219232 5 5 9]
 [0.7889556659397083 3 3 -1]]


### Visualise Discord and Motif Pair ###

In [12]:
mp = mp_orig

def visualise_discord(mp):
    # the global maximum from the matrix profile correspond to the discord
    discord_idx = np.argsort(mp[:, 0])[-1] # startindex of discord with length m

    ## Visualize Discord ##
    fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0})
    plt.suptitle('Discord (Anomaly/Novelty) Discovery', fontsize='30')

    axs[0].plot(ts)
    axs[0].set_ylabel('Sample TS', fontsize='20')
    rect = Rectangle((discord_idx, 0), window_size, np.amax(ts), facecolor='lightgrey')
    axs[0].add_patch(rect)
    axs[1].set_xlabel('Time', fontsize ='20')
    axs[1].set_ylabel('Matrix Profile', fontsize='20')
    axs[1].axvline(x=discord_idx, linestyle="dashed")
    axs[1].plot(mp[:, 0])
    plt.show()
def visualise_motif_pair(mp) -> None:
    # the global minima from the matrix profile correspond to the locations of the two subsequences that make up the motif pair
    motif_idx = np.argsort(mp[:, 0])[0] # startindex of motif with length m
    motif_nearest_neighbor_idx = mp[motif_idx, 1] # startindex of motif with length m

    ## Visualize Motifs ##
    fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0})
    plt.suptitle('Motif (Pattern) Discovery', fontsize='30')

    axs[0].plot(ts)
    axs[0].set_ylabel('Sample TS', fontsize='20')
    rect = Rectangle((motif_idx, 0), window_size, np.amax(ts), facecolor='lightgrey')
    axs[0].add_patch(rect)
    rect = Rectangle((motif_nearest_neighbor_idx, 0), window_size, np.amax(ts), facecolor='lightgrey')
    axs[0].add_patch(rect)
    axs[1].set_xlabel('Time', fontsize ='20')
    axs[1].set_ylabel('Matrix Profile', fontsize='20')
    axs[1].axvline(x=motif_idx, linestyle="dashed")
    axs[1].axvline(x=motif_nearest_neighbor_idx, linestyle="dashed")
    axs[1].plot(mp[:, 0])
    plt.show()

visualise_discord(mp)
visualise_motif_pair(mp)

NameError: name 'mp_orig' is not defined